In [1]:
!pip install -q git+https://github.com/tensorflow/docs

In [9]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os


In [10]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1000

In [11]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [12]:
def build_feature_extractor():
    feature_extractor = tf.keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)


    
    preprocess_input = tf.keras.applications.vgg16.preprocess_input 

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [13]:
train_df = pd.read_csv('train_df.csv')
train_df = train_df.iloc[: , 1:]
test_df = pd.read_csv('test_df.csv')
test_df = test_df.iloc[: , 1:]

In [16]:
import shutil
import glob

path = "/Users/nalthan/Desktop/vertiasopencvprojecy/"
normal_file_names = [path_normal + "/" + filename for filename in os.listdir(path) if filename.endswith('.mp4')]

'''for ind in train_df.index:
    from IPython.core.debugger import Pdb; Pdb().set_trace()
    if train_df.loc[ind, "frame_50"].where() == 1:
        name = train_df.loc[ind, "vidname"]
        length = 6 -len(name)
        name = name.zfill(length)
        print(name)
        os.rename(path+"Crash-1500/"+name+".mp4", path+"train/"+name+".mp4")
    if train_df.loc[ind, "frame_50"].item() == 0:
        name = train_df.loc[ind, "vidname"]
        print(name)
        os.rename(path+"Normal/"+name+".mp4", path+"train/"+name+".mp4")'''
'''for f in glob.glob(path+"train"):
    print(f)
    os.remove(f)
for f in glob.glob(path+"test"):
    print(f)
    os.remove(f)'''

for row in train_df.iterrows():
    if str(row[1][0]) == ".DS_Store":
        continue
    name = str(row[1][0])
    name = name.zfill(6)
    if row[1][1] == 1:
        shutil.copy(path+"Crash-1500/"+name+".mp4", path+"train/"+name+".mp4")
    else:
        shutil.copy(path+"Normal/"+name+".mp4", path+"train/"+name+".mp4")
        
for row in test_df.iterrows():
    name = str(row[1][0])
    name = name.zfill(6)
    if row[1][1] == 1:
        shutil.copy(path+"Crash-1500/"+name+".mp4", path+"test/"+name+".mp4")
    else:
        shutil.copy(path+"Normal/"+name+".mp4", path+"test/"+name+".mp4")

In [17]:
print(train_df)


label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=tf.convert_to_tensor(np.unique(train_df["frame_50"].astype(str).tolist())))

print(label_processor.get_vocabulary())


      vidname  frame_50
0         106         1
1        1139         1
2        2684         0
3        1657         0
4        1479         0
...       ...       ...
3036      439         0
3037      640         1
3038     2789         0
3039      745         0
3040      324         0

[3041 rows x 2 columns]
['0', '1']


/Users/nalthan/anaconda3/lib/python3.11/site-packages/numpy/core/numeric.py:2468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["vidname"].values.tolist()
    labels = df["frame_50"].values

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        print(idx, path)
        print(type(path))
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, str(path).zfill(6)+'.mp4'))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "/Users/nalthan/Desktop/vertiasopencvprojecy/train")
test_data, test_labels = prepare_all_videos(test_df, "/Users/nalthan/Desktop/vertiasopencvprojecy/test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

In [19]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/10
64/67 [===========================>..] - ETA: 0s - loss: 0.5966 - accuracy: 0.7700
Epoch 1: val_loss improved from inf to 0.49881, saving model to /tmp/video_classifier
67/67 [==============================] - 5s 33ms/step - loss: 0.5907 - accuracy: 0.7726 - val_loss: 0.4988 - val_accuracy: 0.7864
Epoch 2/10
64/67 [===========================>..] - ETA: 0s - loss: 0.5190 - accuracy: 0.7871
Epoch 2: val_loss improved from 0.49881 to 0.48061, saving model to /tmp/video_classifier
67/67 [==============================] - 1s 9ms/step - loss: 0.5193 - accuracy: 0.7867 - val_loss: 0.4806 - val_accuracy: 0.7864
Epoch 3/10
64/67 [===========================>..] - ETA: 0s - loss: 0.5169 - accuracy: 0.7886
Epoch 3: val_loss improved from 0.48061 to 0.46569, saving model to /tmp/video_classifier
67/67 [==============================] - 1s 9ms/step - loss: 0.5191 - accuracy: 0.7867 - val_loss: 0.4657 - val_accuracy: 0.7864
Epoch 4/10
64/67 [===========================>..] - ETA: 0s - lo

In [20]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    return [round(probabilities[0]*100, 2), round(probabilities[1]*100, 2)] 



test_video = str(np.random.choice(test_df["vidname"].values.tolist())).zfill(6)
print(f"Test video path: {test_video}")
#test_frames = sequence_prediction(str(test_video)+".mp4")


Test video path: 000827


In [21]:
vidname = []
real = []
predict0 = []
predict1 = []

for index, row in test_df.iterrows():
    vidname.append(str(row["vidname"]).zfill(6))
    real.append(row["frame_50"])
    percentages = []
    percentages = sequence_prediction(str(row["vidname"]).zfill(6)+".mp4")
    predict0.append(percentages[0])
    predict1.append(percentages[1])

[ WARN:0@10132.995] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@10132.995] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin3574 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@10132.995] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@10132.995] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


1/1 [==============================] - 1s 863ms/step


[ WARN:0@10136.579] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@10136.579] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin3575 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@10136.579] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@10136.579] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


1/1 [==============================] - 0s 7ms/step


[ WARN:0@10139.161] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@10139.161] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin3576 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@10139.161] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@10139.161] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


...

1/1 [==============================] - 0s 7ms/step


[ WARN:0@11588.049] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@11588.049] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin4105 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@11588.049] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@11588.049] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


1/1 [==============================] - 0s 7ms/step


[ WARN:0@11590.788] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@11590.788] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin4106 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@11590.788] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@11590.788] global /Users/cbousseau/work/recipes/ci_py311/opencv-suite_1677933071594/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


1/1 [==============================] - 0s 7ms/step


In [22]:
dict = {'vidname': vidname, 'real': real, 'predict0': predict0, 'predict1':predict1} 
    
results = pd.DataFrame(dict)
results.to_csv('vgg_results.csv')